In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 1
part = 0
N_files = 4
N_loc_files = 2

In [3]:
N_centr = 5
N_pt = 1
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,100,0,20,20,40,40,60,60,80,80,100]
pt_boarders = [0.5,1.5,1.0,2.0,0,5.0,1.0,5.0,0.5,5]
file_path="../sim/output/invmass_embed/"
file_names=["ccbar_v3_12M.root","jpsi_v5_10M.root","bbbar_v0_10M.root","phi_v2_10M.root","photon_v2_50M.root"]
part_names = ["c#bar{c}","J/#psi","b#bar{b}","#phi","#gamma#to ee"]
hist_mass_DCA_names = ["inv_mass_dca_fg5","inv_mass_dca_fg7"]
hist_DCA2_names = ["DCA2_hist_0","DCA2_hist_4"]
hist_DCA12_names=["DCA12_hist_0","DCA12_hist_4"]
hists_all = [hist_mass_DCA_names,hist_DCA2_names,hist_DCA12_names]
N_type = len(hists_all)

In [4]:
hists_read = []

for iFile in range(N_files):
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_emc_read0 = []
    for itype in range(N_type):
        hist_select_3D0 = []
        for ihist in range(len(hists_all[itype])):
            hist_select_3D0.append(infile.Get(hists_all[itype][ihist]))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
            hist_select_3D0[-1].SetName(hist_select_3D0[-1].GetName()+f"{iFile}")
        hists_emc_read0.append(hist_select_3D0)
    hists_read.append(hists_emc_read0)

infile.Close()

In [5]:
type_loc =1
N_dca_files = 4
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",1400,450*1)
legends = []
c0.Divide(2,1)
for itype in range(N_type_loc):
    legends.append(root.TLegend(0.65,0.5,0.91,0.88))
    Format_Legend(legends[-1],0.03)
    for iFile in range(0,N_dca_files):
        c0.cd(itype+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(0.5)
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(2.5)
        project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",1,4,ibin0,ibin1)
        Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  0, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1-0.1*iFile)
        project.GetXaxis().SetRange(project.FindBin(-500),project.FindBin(500))
        #root.gPad.SetLogy()
        if iFile == 0 :
            project.Draw("H")
            max = project.GetMaximum()
        else:
            project.Scale(max/project.GetMaximum())          
            project.Draw("H same")
        legends[-1].AddEntry(project,part_names[iFile],"l")
    legends[-1].Draw()


c0.Draw()
c0.SaveAs(f"output/single_dca.png")

In [6]:
ipt = 2
type_loc =0
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c1",f"c1",1400,450*1)
c0.Divide(2,1)
N_dca_files = 4
legends = []
for itype in range(N_type_loc):
    max = 0

    legends.append(root.TLegend(0.65,0.5,0.91,0.88))
    Format_Legend(legends[-1],0.03)
    for iFile in range(0,N_dca_files):
        c0.cd(itype+1)
        ibinY0 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(1.0)
        ibinY1 = hists_read[iFile][type_loc][itype].ProjectionY().FindBin(5.9)
        ibinZ0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibinZ1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        project = hists_read[iFile][type_loc][itype].ProjectionX(hists_read[iFile][type_loc][itype].GetName()+"_X",ibinY0,ibinY1,ibinZ0,ibinZ1)
        Format_Hist_total(project, "DCA_{T} #mum",  "dN/dDCA_{T}, 1/20 #mum^{-1}", 0.15, 0.15, 0, 0,  .07,  0.06, 21,  0, iFile+1,  7,  iFile+1,  1, 1.1, "",  1,  1-0.1*iFile)
        project.GetXaxis().SetRange(project.FindBin(1),project.FindBin(1000))
        root.gPad.SetLogy()
        if iFile == 0 :
            project.Draw("H")
            max = project.GetMaximum()
        else:
            project.Scale(max/project.GetMaximum())          
            project.Draw("H same")
        legends[-1].AddEntry(project,part_names[iFile],"l")
    legends[-1].Draw()


c0.Draw()
c0.SaveAs(f"output/dca2.png")

In [7]:
ipt = 0
type_loc = 0 
N_dca_files = 4
N_type_loc= len(hists_all[type_loc])
c0 = root.TCanvas(f"c0",f"c0",1400,450*N_dca_files)
c0.Divide(N_type_loc,N_dca_files)
for itype in range(N_type_loc):
    for iFile in range(0,N_dca_files):
        c0.cd(itype*N_dca_files+iFile+1)
        ibin0 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+0])
        ibin1 = hists_read[iFile][type_loc][itype].ProjectionZ().FindBin(pt_boarders[2*ipt+1])
        #hists_read[iFile][type_loc][itype].GetZaxis().SetRange(ibin0,ibin1)
        hists_read[iFile][type_loc][itype].Project3D("yx").Draw("colz")

c0.Draw()
#c0.SaveAs(f"output/rich_emcal_qa/qa_pt_centr_{ipt}.png")